## google movie 에서 영화 정보 가져오기

In [20]:
import requests
from bs4 import BeautifulSoup

In [3]:
url = "https://play.google.com/store/movies/top"
res = requests.get(url) 
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml")

In [4]:
movies = soup.find_all('div', attrs = {'class': 'ImZGtf mpg5gc'})
print(len(movies))

0


In [6]:
with open("movie.html", "w", encoding="utf-8") as f:
    #f.write(res.text)
    f.write(soup.prettify()) # html 을 좀 더 예쁘게 출력해줌
    # 접속하는 헤더정보에 따라 서로 다른 정보를 반환함

### 접속하는 헤더정보에 따라 서로 다른 정보를 반환함
- 따라서 headers를 통해 user agent를 달리 받아와야됨

In [15]:
headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36",
    "Accept-Language":"ko-KR,ko" # 이렇게 써줘야 한글 페이지를 반환해줌
}

url = "https://play.google.com/store/movies/top"
res = requests.get(url, headers=headers) 
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml")

In [16]:
movies = soup.find_all('div', attrs = {'class': 'ImZGtf mpg5gc'})
print(len(movies))

10


In [17]:
with open("movie.html", "w", encoding="utf-8") as f:
    #f.write(res.text)
    f.write(soup.prettify()) # html 을 좀 더 예쁘게 출력해줌
    # 접속하는 헤더정보에 따라 서로 다른 정보를 반환함

### 왜 10개만 뜨지???
- 스크롤을 내릴때 마다 새롭게 로딩
- 이런걸 "동적 페이지"라고 함
- requests를 통해서 접속했을때, 최초 10개만 보여짐
- 나머지는 ?
- 이럴때 필요한게 selenium

In [21]:
for movie in movies:
    title = movie.find('div', attrs={'class':'WsMG1c nnK0zc'}).get_text()
    print(title)

원더우먼 1984
테넷
미션 파서블
성판17: 남자들의 17가지 성적 판타지
너의 이름은. (자막판)
어벤져스 : 엔드게임 (자막판)
날씨의 아이 (자막)
해리포터와 마법사의 돌
조커
겨울왕국 2 (더빙판)


## selenium을 이용한 동적 페이지 웹 스크래핑

In [1]:
from selenium import webdriver

In [2]:
browser = webdriver.Chrome()
browser.maximize_window()

url = "https://play.google.com/store/movies/top"
browser.get(url)

### 스크롤 내리기

In [6]:
# 지정한 위치까지 스크로 내리기
browser.execute_script("window.scrollTo(0, 1600)") # 2560 x 1600

In [13]:
# 화면 가장 아래로 스크린 내리기
browser.execute_script("window.scrollTo(0, document.body.scrollHeight)") # 문서의 가장 아래쪽으로 스크롤 내리기

In [17]:
import time

interval = 2 # 2초에 한번씩 스크롤 내리기
prev_height = browser.execute_script("return document.body.scrollHeight")

In [18]:
while True:
    # 스크롤 가장 아래로 내림
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    
    # 2초 기다림
    time.sleep(interval)

    # 높이 체크
    curr_height = browser.execute_script("return document.body.scrollHeight")
    if curr_height == prev_height:
        break
    
    prev_height = curr_height

print('스크롤 완료')

스크롤 완료


### 이제 스크래핑 작업을 하면 됨

In [25]:
# selenium으로 이미 페이지 소스를 가져왔으니 url이나 header같은거 필요 없다

soup = BeautifulSoup(browser.page_source, "lxml")

# movies = soup.find_all('div', attrs = {'class': ['ImZGtf mpg5gc', 'Vpfmgd']})
# 10개가 넘어가니까 형식이 좀 달라서 두개 다 가져오기로 함
# 근데 'Vpfmgd'클래스가 위에 10개도 붙어있어서 굳이 리스트로 안하고 밑에 처럼 하면 됨
movies = soup.find_all('div', attrs = {'class': 'Vpfmgd'})


print(len(movies))


200


In [44]:
for movie in movies:
    title = movie.find('div', attrs={'class':'WsMG1c nnK0zc'}).get_text()
    link = movie.find('a', attrs={'class':'JC71ub'})['href']
    # 할인 된 영화만 뽑기
    if movie.find('span', attrs={'class':'SUZt4c djCuy'}):
        print(title)
        print("할인 전 가격 : ", movie.find('span', attrs={'class':'SUZt4c djCuy'}).get_text())
        print("할인 된 가격 : ", movie.find('span', attrs={'class':'VfPpfd ZdBevf i5DZme'}).get_text())
        print("링크 : ", "https://play.google.com" + link)
        print()

browser.quit()

주짓수
할인 전 가격 :  ₩10,000
할인 된 가격 :  ₩5,000
링크 :  https://play.google.com/store/movies/details/%EC%A3%BC%EC%A7%93%EC%88%98?id=NDGwdH9cCCk.P

조제
할인 전 가격 :  ₩11,000
할인 된 가격 :  ₩5,000
링크 :  https://play.google.com/store/movies/details/%EC%A1%B0%EC%A0%9C?id=FiwG10w0r7o.P

도굴
할인 전 가격 :  ₩7,700
할인 된 가격 :  ₩5,500
링크 :  https://play.google.com/store/movies/details/%EB%8F%84%EA%B5%B4?id=HQcgbruj254.P

스푸트닉
할인 전 가격 :  ₩7,000
할인 된 가격 :  ₩5,000
링크 :  https://play.google.com/store/movies/details/%EC%8A%A4%ED%91%B8%ED%8A%B8%EB%8B%89?id=qQxeFGNR12U.P

뱅가드
할인 전 가격 :  ₩10,000
할인 된 가격 :  ₩4,000
링크 :  https://play.google.com/store/movies/details/%EB%B1%85%EA%B0%80%EB%93%9C?id=cj16bDavhgI.P

해리포터 시리즈 완결 패키지
할인 전 가격 :  ₩36,000
할인 된 가격 :  ₩18,900
링크 :  https://play.google.com/store/movies/details/%ED%95%B4%EB%A6%AC%ED%8F%AC%ED%84%B0_%EC%8B%9C%EB%A6%AC%EC%A6%88_%EC%99%84%EA%B2%B0_%ED%8C%A8%ED%82%A4%EC%A7%80?id=DsVgRu5dDdY

검객
할인 전 가격 :  ₩5,000
할인 된 가격 :  ₩2,500
링크 :  https://play.google.com/store/movies/details